In [49]:
from google.colab import drive

drive.mount('/content/drive')
!ls '/content/drive/My Drive/DSL Final Project/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
'Blog Post'
'Blog Post.gdoc'
 body_parts.txt
 Classifier.ipynb
'Copy of Models.ipynb'
 data_for_NN.pickle
 data_nouns.pickle
 data.pickle
'Data Science Presentation.gslides'
 featured_data.pickle
'List of users and number of posts.gdoc'
'List of users and number of posts.txt'
'most_common_nouns (1).gdoc'
 most_common_nouns.gdoc
 most_common_nouns.txt
 Planning.gdoc
'Predicting Popular Reddit Posts.ipynb'
 Scraper.ipynb
 supernaturalwords.txt
 trainer.ipynb
 violentwords.txt


In [0]:
# !pip install grammar-check
# import grammer_check
import urllib.request, json 
import pandas as pd
import datetime
import time
import numpy as np


In [0]:
# dictionary = dict()
# dictionary['author']=[]
# dictionary['created_utc']=[]
# dictionary['post_content']=[]
# dictionary['is_series']=[]
# dictionary['title']=[]
# dictionary['num_upvotes']=[]
# url1 = 'https://api.pushshift.io/reddit/submission/search/?after='
# url2 = '&sort_type=score&sort=desc&limit=1000&subreddit=NoSleep'
# start = 1417392000
# count=0
# for i in range(1826):
#     time.sleep(0.5)
#     end = start+86400
#     url = url1+str(start)+'&before='+str(end)+url2
#     with urllib.request.urlopen(url) as u:
#         data = json.loads(u.read().decode())
#         for d in data['data']:
#             count+=1
#             print(count)          
#             dictionary['created_utc'].append(d['created_utc'])
#             dictionary['title'].append(d['title'])
#             dictionary['author'].append(d['author'])
#             if 'link_flair_text' in d.keys():
#               dictionary['is_series'].append(d['link_flair_text'])
#             else:
#               dictionary['is_series'].append('')
#             if 'selftext' in d.keys():
#                 dictionary['post_content'].append(d['selftext'])
#             else:
#                 dictionary['post_content'].append('')
#             dictionary['num_upvotes'].append(d['score'])
#             post_id = d['id']
#     start = end

In [0]:
# df = pd.DataFrame.from_dict(dictionary)
# df.to_pickle('/content/drive/My Drive/DSL Final Project/data.pickle')

In [0]:
df = pd.read_pickle('/content/drive/My Drive/DSL Final Project/data.pickle')
#df = pd.read_pickle('data.pickle')
#Removes announcement posts, redacted posts
print(df.shape[0])
df = df[~df.title.str.contains("Congratulations!")]
print(df.shape[0])
df = df[~df.title.str.contains("Voting Thread")]
print(df.shape[0])
df = df[~df.title.str.contains("Redacted")]
df = df[~df.title.str.contains("redacted")]
df = df[~df.title.str.contains("REDACTED")]
df = df[~df.title.str.contains("Contest Winner")]
print(df.shape[0])
df["post_content"].replace("", np.nan, inplace=True)
df["title"].replace("", np.nan, inplace=True)
df.dropna(subset=["post_content"], inplace=True)
df.dropna(subset=["title"], inplace=True)
# df = df[~df.post_content.str.equals("")]
print(df.shape[0])

202398
202380
202323
202245
188822


In [0]:
#Feature 1: Story Wordcount, Feature 2: Title Wordcount
df['story_wordcount'] = df['post_content'].apply(lambda x: len(x.split()))
df["title_wordcount"] = df['title'].apply(lambda x: len(x.split()))
#Feature 3: Time of day
df["time_of_day"] = df["created_utc"].apply(lambda x: datetime.datetime.utcfromtimestamp(x).strftime("%H"))
#Feature 4: Month posted
df["month_posted"] = df["created_utc"].apply(lambda x: datetime.datetime.utcfromtimestamp(x).strftime("%m"))
#Feature 5: Year posted
df["year_posted"] = df["created_utc"].apply(lambda x: datetime.datetime.utcfromtimestamp(x).strftime("%y"))
#Feature 6: Day of week
df["day_of_week"] = df["created_utc"].apply(lambda x: datetime.datetime.utcfromtimestamp(x).isoweekday())
def capitalized_title(x):
    x = x.split()
    num_words = len(x)
    num_caps = 0
    for word in x:
        if(word[0].istitle()):
            num_caps = num_caps + 1
    if (float(num_caps)/num_words >= 0.5):
      return 1
    else:
      return 0
def has_period(x):
    x = x.replace("...", " ")
    x = x.replace("....", " ")
    if ("." in x):
        return 1
    else:
        return 0
def all_caps(x):
    x = x.split()
    num_words = len(x)
    num_all_caps = 0
    for word in x:
        if(word.isupper()):
            continue
        else:
            return 0
    return 1
#Feature: % of post that is dialogue
def QuotedWords(story):
    content = story.split()
    quotedWords = 0
    inQuote = False
    total_words = 0
    for word in content:
        total_words += 1
        if ((word.find('\"') != -1) & (inQuote == False)):
            quotedWords += 1
            inQuote = True
        elif ((inQuote == True) & (word.find('\"')) != -1):
            inQuote = False
        elif (inQuote):
            quotedWords += 1
    if total_words == 0:
        return 0
    return (float(quotedWords)/total_words)*100.0
df["%_in_quotes"] = df['post_content'].apply(lambda x: QuotedWords(x))
#Feature: % of post that is bold
def BoldedWords(story):
    content = story.split()
    boldedWords = 0
    inQuote = False
    total_words = 0
    for word in content:
        total_words += 1
        if ((word.find("__") != -1) & (word.find("**") != -1) & (inQuote == False)):
            boldedWords += 1
            inQuote = True
        elif inQuote & (word.find("__") != -1) & (word.find("**") != -1):
            inQuote = False
        elif (inQuote):
            boldedWords += 1
    if total_words == 0:
        return 0
    return (float(boldedWords)/total_words) * 100.0
df["%_in_bold"] = df['post_content'].apply(lambda x: BoldedWords(x))
def number_of_paragraphs(x):
    return (len(x.split('\n')))
def len_of_title(x):
    return (len(x.split()))
def has_elipses(x):
    if("..." in x):
      return 1
    else:
      return 0
def has_question_mark(x):
    if("?" in x):
      return 1
    else:
      return 0


In [0]:
def violent_word_count(x):
    x = x.split()
    f = open('/content/drive/My Drive/DSL Final Project/violentwords.txt', 'r')
    violent_words = [line.split(',') for line in f.readlines()]
    #violent_words = filter(lambda a: a!='\n', temp)
    f.close()
    violent_word_count = 0
    for word in x:
      if (word.lower() in violent_words[0]):
          violent_word_count = violent_word_count + 1
    return violent_word_count
def bodyparts_word_count(x):
    x = x.split()
    f = open('/content/drive/My Drive/DSL Final Project/body_parts.txt', 'r')
    bodyparts_words = [line.split(',') for line in f.readlines()]
    #bodyparts_words = filter(lambda a: a!='\n', temp)
    f.close()
    bodyparts_word_count = 0
    #print(bodyparts_words)
    for word in x:
      if (word.lower() in bodyparts_words[0]):
          bodyparts_word_count = bodyparts_word_count + 1
    return bodyparts_word_count
def supernat_word_count(x):
    x = x.split()
    f = open('/content/drive/My Drive/DSL Final Project/supernaturalwords.txt', 'r')
    supernat_words = [line.split(',') for line in f.readlines()]
    #supernat_words = filter(lambda a: a!='\n', temp)
    f.close()
    supernat_word_count = 0
    for word in x:
      if (word.lower() in supernat_words[0]):
          supernat_word_count = supernat_word_count + 1
    return supernat_word_count


# tool = grammar_check.LanguageTool('en-GB')
# def grammar_mistake_count(x):
#     return len(tool.check(x))

In [0]:
df.sort_index(inplace=True)
df["capitalize_title"] = df["title"].apply(lambda x: capitalized_title(x))
df["has_period"] = df["title"].apply(lambda x: has_period(x))
df["all_caps"] = df["title"].apply(lambda x: all_caps(x))
df["number_of_paragraphs"] = df["post_content"].apply(lambda x: number_of_paragraphs(x))
df["len_of_title"] = df["title"].apply(lambda x: len_of_title(x))
df["has_elipses"] = df["title"].apply(lambda x: has_elipses(x))
df["has_question_mark"] = df["title"].apply(lambda x: has_question_mark(x))
# added features: (Hadar)
df['violent_count'] = df['post_content'].apply(lambda x: violent_word_count(x))
df['bodyparts_count'] = df['post_content'].apply(lambda x: bodyparts_word_count(x))
df['supernat_count'] = df['post_content'].apply(lambda x: supernat_word_count(x))
#df['grammar_mistake_count'] = df['content'].apply(lambda x: grammar_mistake_count(x))

In [0]:

# save df as a pickle file 
#df.to_pickle('/content/drive/My Drive/DSL Final Project/featured_data.pickle')


In [0]:
df = pd.read_pickle('/content/drive/My Drive/DSL Final Project/featured_data.pickle')

In [0]:
print(bodyparts_word_count(df.loc[0, 'post_content']))

print(supernat_word_count(df.loc[3, 'post_content']))
print(violent_word_count(df.loc[0, 'post_content']))

In [0]:
username_list = df.author.unique()
times_posted = {}
for user in username_list:
    times_posted[user] = (df.author == user).sum()


In [0]:
import numpy as np
with open('/content/drive/My Drive/DSL Final Project/List of users and number of posts.txt','r') as inf:
    dict_from_file = eval(inf.read().strip())
times_posted = dict_from_file
def get_times_posted(x):
    if(times_posted[x["author"]] <= 1):
        return 0
    if(x["author"] == '[deleted]'):
      return np.nan
    else:
        df_temp = df[df['author'] == x['author']]
        return((df_temp.created_utc < x['created_utc']).sum())
df['times_posted'] = 

SyntaxError: ignored

In [0]:
print(times_posted)

{'xcf3131': 3, 'Renner587': 5, 'Ibitemynails': 1, 'namedghost': 4, 'Hannah_Bardelli': 1, 'blackeyedkids': 1, 'Vanished_City': 1, '[deleted]': 38185, 'JakeIsBack': 1, 'rachelanderson': 2, 'KateReads': 1, 'funnycorgi': 1, 'LonelyOffice': 1, 'UmbraLox-': 1, 'UpdateYourselfAdobe': 2, 'theotherghostgirl': 24, 'TheSylvaBullet': 1, 'Syshana': 6, 'fallingstar9': 3, 'liveandletlive177': 1, 'dwilljohnsonp2': 1, 'Ryuary': 11, 'texas_shiner': 1, 'Sinazinha': 10, 'janedoe_467': 1, 'sdaly0107': 1, 'Choupette8': 1, 'Mr_Shiver86': 2, 'Luv2LuvEm1': 1, 'zeecaptien': 1, 'Ink184': 1, 'Grindhorse': 16, 'KoRax2667': 1, 'MatthewJacobson': 1, 'youngsterjose42': 12, 'one_seventy_five': 2, 'cynicallyspoken': 4, 'SupposedlyanAtheist': 2, 'McMiron': 1, 'WalkingAmongDevils': 1, 'InsideMedusa': 1, 'Shigidishmo': 1, 'thats_a_trade_secret': 1, 'Internenkana': 1, 'nosleep2nite4u': 1, 'R_rites': 1, 'Peow_peow': 10, 'wixp': 2, 'TDoP93': 1, 'alreadyawesome': 6, 'bepstein90': 1, 'Fozzybear513': 1, 'jetheather': 1, 'richar

In [0]:
df.head()

,author,created_utc,post_content,is_series,title,num_upvotes
0,xcf3131,1417407137,It's a strange feeling when your childhood hom...,,There was a murder at my childhood home this week,1978
1,Renner587,1417466306,I sort of got a cat recently. It’s not really ...,,The mysterious thing inside the fireplace,1494
2,Ibitemynails,1417401469,**[daydalia](http://www.reddit.com/user/daydal...,,Your November NoSleep Contest Winner is...,480
3,namedghost,1417469526,namedghost&gt;&gt; I clicked it.\n\nredacted&g...,,black²22▓»ª*È.,420
4,Hannah_Bardelli,1417466215,[So someone decided to be funny and post this ...,,"Alright, punks, this isn’t funny anymore",195


In [45]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.tokenize import word_tokenize
from nltk import pos_tag
one_word_freq_nouns = {}
counter = 0
def get_noun_freq(x):
  global counter
  counter += 1
  if(counter%10000 == 0):
    print(str(counter))
  text = word_tokenize(x)
  text = pos_tag(text)
  for word in text:
    word = (word[0].lower(), word[1])
    if((word[1] == 'NN') or (word[1] == 'NNS') or (word[1] == 'NNP') or (word[1] == 'NNPS')):
      if (word[0] in one_word_freq_nouns.keys()):
        one_word_freq_nouns[word[0]] = one_word_freq_nouns[word[0]] + 1
      else:
        one_word_freq_nouns[word[0]] = 1
df['title'].apply(lambda x: get_noun_freq(x))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000


0         None
1         None
3         None
4         None
5         None
          ... 
202393    None
202394    None
202395    None
202396    None
202397    None
Name: title, Length: 188822, dtype: object

In [0]:
one_word_freq_nouns = [(k, v) for k, v in one_word_freq_nouns.items()] 
def Sort_Tuple(tup):  
  
    # reverse = None (Sorts in Ascending order)  
    # key is set to sort using second element of  
    # sublist lambda has been used  
    return(sorted(tup, key = lambda x: x[1])) 
one_word_freq_nouns = Sort_Tuple(one_word_freq_nouns)
one_word_freq_nouns = one_word_freq_nouns[::-1]

In [29]:
print(one_word_freq_nouns[:100])

[('part', 22063), (']', 7714), ('[', 4715), ('night', 4691), ('house', 4528), ('something', 4020), ('man', 3961), ('story', 3942), ('my', 3807), ('friend', 3098), ('me', 2556), ('life', 2523), ('nsfw', 2297), ('help', 2288), ('someone', 2132), ('time', 2094), ('’', 1958), ('woods', 1900), ('home', 1858), ('update', 1792), ('things', 1758), ('day', 1748), ('a', 1742), ('girl', 1581), ('t', 1543), ('people', 1540), ('years', 1532), ('stories', 1524), ('dream', 1517), ('room', 1445), ('town', 1376), ('pt', 1358), ('death', 1351), ('final', 1287), ('experience', 1256), ('ghost', 1253), ('door', 1250), ('please', 1209), ('dark', 1204), ('school', 1178), ('i', 1140), ('game', 1122), ('family', 1104), ('thing', 1102), ('sister', 1075), ('hell', 1073), ('anyone', 1050), ('sleep', 1047), ('new', 1045), ('dreams', 1037), ('brother', 1011), ('one', 1003), ('world', 946), ('eyes', 945), ('nightmare', 945), ('dog', 942), ('friends', 941), ('demon', 892), ('mother', 882), ('s', 849), ('horror', 849)

In [0]:
with open('/content/drive/My Drive/DSL Final Project/most_common_nouns.txt', 'w') as filehandle:
    for listitem in one_word_freq_nouns:
        filehandle.write(str(listitem) + "\n")

In [47]:
one_word_freq_nouns.remove((']', 7714))
one_word_freq_nouns.remove(('[', 4715))
one_word_freq_nouns.remove(('’', 1958))
one_word_freq_nouns.remove(('t', 1543))
one_word_freq_nouns.remove(('i', 1140))
def contains_word(x, input_word):
  if (input_word in x):
    return 1
  else:
    return 0
df_nouns = pd.DataFrame()
for word in one_word_freq_nouns[:50]:
  df_nouns[word[0]] = df["title"].apply(lambda x: contains_word(x, word[0]))
df_nouns.head()

,part,night,house,something,man,story,my,friend,me,life,nsfw,help,someone,time,woods,home,update,things,day,a,girl,people,years,stories,dream,room,town,pt,death,final,experience,ghost,door,please,dark,school,game,family,thing,sister,hell,anyone,sleep,new,dreams,brother,one,world,eyes,nightmare
0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
df_nouns.to_pickle('/content/drive/My Drive/DSL Final Project/data_nouns.pickle')

In [0]:
from sklearn.linear_model import Lasso
import pandas as pd
import numpy as np
import pickle
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [65]:
file = open('/content/drive/My Drive/DSL Final Project/featured_data.pickle', 'rb')
df = pickle.load(file)
df = df.reset_index()
df = df.drop(['index'],axis=1)
dropped = []
for i in range(len(df)):
  if df.iloc[i,3]==None or 'Removed' in df.iloc[i,3]:
    dropped.append(i)
df = df.drop(dropped)
df = df.reset_index()
df = df.drop(['index'],axis=1)
flairs = pd.get_dummies(df['is_series'])
df["time_of_day"] = df["time_of_day"].apply(lambda x: "time_of_day: " + str(x))
time_of_day = pd.get_dummies(df['time_of_day'])
df["day_of_week"] = df["day_of_week"].apply(lambda x: "day_of_week: " + str(x))
day_of_week = pd.get_dummies(df['day_of_week'])
df["year_posted"] = df["year_posted"].apply(lambda x: "20" + str(x))
year_posted = pd.get_dummies(df['year_posted'])
df["month_posted"] = df["month_posted"].apply(lambda x: "month_posted: " + str(x))
month_posted = pd.get_dummies(df['month_posted'])
df = df.drop(columns=['author','created_utc','post_content','is_series','title','time_of_day','month_posted','year_posted','day_of_week'])
df = pd.concat([df,flairs,time_of_day,day_of_week,year_posted,month_posted],axis=1)
#df = pd.concat([df.iloc[:,0],time_of_day,day_of_week],axis=1)
#df = pd.concat([df['num_upvotes'],flairs],axis=1)
df_time = pd.DataFrame()
times = ["time_of_day: " + str(i).zfill(2) for i in range(24)]
days = ["day_of_week: " + str(i) for i in range(1,8)]
for day in days:
  for time in times:
    df_time[day + "," + time] = df[day]+df[time]
df_time = df_time.replace(1,0)
df_time = df_time.replace(2,1)
df_time = pd.concat([df.iloc[:,0],df_time],axis=1)
df = df_time
df=pd.concat([df['num_upvotes'],df['len_of_title']],axis=1)

KeyError: ignored

In [70]:
df_nouns = pd.read_pickle('/content/drive/My Drive/DSL Final Project/data_nouns.pickle')
df = df.reset_index()
df = df.drop(['index'],axis=1)
dropped = []
for i in range(len(df)):
  if df.iloc[i,3]==None or 'Removed' in df.iloc[i,3]:
    dropped.append(i)
df = df.drop(dropped)
df = df.reset_index()
df = df.drop(['index'],axis=1)
df = pd.concat([df["num_upvotes"], df_nouns],axis=1)

data = df.values
y = data[:,0]
y = y[:,np.newaxis]
X = df_nouns.values
scalerX = StandardScaler()
scalery = StandardScaler()
scaled_X = scalerX.fit_transform(X)
scaled_y = scalery.fit_transform(y)
X_train, X_test, y_train, y_test = train_test_split(scaled_X, scaled_y, test_size=0.5, random_state=42)

lasso = Lasso(alpha=0.001).fit(X_train,y_train)
lasso.score(X_test,y_test)

coef = pd.Series(lasso.coef_, index = df.columns[1:])
print("Lasso picked " + str(sum(coef != 0)) + " variables and eliminated the other " +  str(sum(coef == 0)) + " variables")
imp_coef = pd.concat([coef.sort_values().head(3),
                     coef.sort_values().tail(0)])

imp_coef.plot(kind = "barh")
plt.title("Coefficients in the Lasso Model")

TypeError: ignored

In [57]:
data

array([['xcf3131', 1417407137,
        "It's a strange feeling when your childhood home is named as the site of a brutal double murder in a national newspaper. This is the feeling I experienced earlier this week. The article in question described how the mother had been killed while watching television at around 10:30 pm. Her assailant had presumably crept up behind her with a knife, as there were no signs of a struggle save for the slit across her neck. The perpetrator had then moved upstairs and killed the three year old daughter in her bedroom. *My* bedroom. The father claimed to have come home from the pub that evening just before 11:00 pm, and then phoned the police at about midnight. He attributed the hour delay between arriving home and calling the authorities to a mixture of shock and incapacity while he vomited after seeing his daughter's body. He had been taken to the police station for questioning. The journalist listed the husband's previous violent offences, and short of o